In [1]:
import numpy as np
import pickle
import time
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as mvn
from scipy.stats import skewnorm
from scipy.stats import norm
from scipy.interpolate import griddata
from pp_mix.src.proto.proto import *
from pp_mix.src.proto.Params import *
from pp_mix.interface import ConditionalMCMC
from pp_mix.params_helper import *

In [2]:
def generate_data(dim):
    if (dim == 1):
        data = np.concatenate([
            np.random.normal(loc = 5, size=100), 
            np.random.normal(loc = -5, size=100)
        ])
        return data.reshape(-1,1)
    else:
        data = np.vstack([
           [mvn.rvs(mean=np.ones(dim) * 5, cov=np.eye(dim)) for _ in range(100)],
           [mvn.rvs(mean=np.ones(dim) * (-5), cov=np.eye(dim)) for _ in range(100)]])
        return data

In [3]:
nrep = 10
dimrange = [1, 2, 3, 4, 5]
strauss_times = np.zeros((nrep, len(dimrange)))

nrange = [5, 10, 20]
dpp_times = np.zeros((nrep, len(dimrange), len(nrange)))

for i in range(nrep):
    print("Running rep: {0}".format(i))
    for j, dim in enumerate([1, 2, 3, 4, 5]):
        data = generate_data(dim)
        if dim == 1:
            #prec_params = GammaParams(alpha=1.0, beta=1.0)
            prec_params = GammaParams()
            prec_params.alpha = 1.0
            prec_params.beta = 1.0
        else:
            # prec_params = WishartParams(
            #     nu=dim+1,identity=True, sigma=3, dim=dim)
            prec_params = WishartParams()
            WishartParams.nu = dim+1
            WishartParams.identity = True
            WishartParams.sigma = 3
            WishartParams.dim = dim
        # gamma_jump_params = GammaParams(alpha=1, beta=1)
        gamma_jump_params = GammaParams()
        gamma_jump_params.alpha = 1
        gamma_jump_params.beta = 1


        strauss_params = make_default_strauss(data)
        strauss_params.fixed_params = False
        strauss_sampler = ConditionalMCMC(
            pp_params=strauss_params,
            prec_params=prec_params,
            jump_params=gamma_jump_params)

        start = time.time()
        strauss_sampler.run(0, 2, 5, data)
        strauss_times[i, j] = time.time() - start

            
        for k, n in enumerate(nrange):
            
            if n == 20 and dim > 3:
                continue
            
            if n == 10 and dim > 4:
                continue
                
            #dpp_params = DPPParams(nu=2.0, rho=3.0, N=n)
            #dpp_params = DPPParams()
            dpp_params = DPPParams()
            dpp_params.nu = 2.0
            dpp_params.rho = 3.0
            dpp_params.N = n
            print(k,n,dim,'dpp_simulation')
            dpp_sampler = ConditionalMCMC(
                pp_params=dpp_params, 
                prec_params=prec_params,
                jump_params=gamma_jump_params)
            start = time.time()
            dpp_sampler.run(0, 2, 5, data)
            dpp_times[i, j, k] = time.time() - start

Running rep: 0
ranges: 
 [[-14.81621093]
 [ 16.5218866 ]]
AdaptiveMetropolis::init()
initialize
 beta: 0.4946056467417964, vol_range: 31.33809753711043, c_star: 15.5
initialize_allocated_means start
initialize_allocated_means <class 'pp_mix.src.point_process.strauss_pp.StraussPP'> --- (4, 1)
initialize done
0 5 1 dpp_simulation
ranges: 
 [[-14.81621093]
 [ 16.5218866 ]]
initialize_allocated_means start
initialize_allocated_means <class 'pp_mix.src.point_process.determinantal_pp.DeterminantalPP'> --- (4, 1)
initialize done
1 10 1 dpp_simulation
ranges: 
 [[-14.81621093]
 [ 16.5218866 ]]
initialize_allocated_means start
initialize_allocated_means <class 'pp_mix.src.point_process.determinantal_pp.DeterminantalPP'> --- (4, 1)
initialize done
2 20 1 dpp_simulation
ranges: 
 [[-14.81621093]
 [ 16.5218866 ]]
initialize_allocated_means start
initialize_allocated_means <class 'pp_mix.src.point_process.determinantal_pp.DeterminantalPP'> --- (4, 1)
initialize done
ranges: 
 [[-14.40393354 -15.232

ValueError: Dimension mismatch: array 'cov' is of shape (0, 0), but 'mean' is a vector of length 2.

In [ ]:
with open("data/execution_times.pickle", "wb") as fp:
    pickle.dump({"dpp": dpp_times, "strauss": strauss_times}, fp)

In [ ]:
with open("data/execution_times.pickle", "rb") as fp:
    times = pickle.load(fp)
    
strauss_times = times["strauss"]
dpp_times = times["dpp"]

In [ ]:
nrange = [5, 10, 20]
dimrange = [1, 2, 3, 4, 5]

plt.plot(dimrange, np.median(strauss_times, axis=0), label="Strauss")
# for k, n in enumerate(nrange):

plt.plot(dimrange[:4], np.median(dpp_times[:, :4, 0], axis=0), label="DPP, N:{0}".format(nrange[0]))
plt.plot(dimrange[:4], np.median(dpp_times[:, :4, 1], axis=0),  label="DPP, N:{0}".format(nrange[1]))
# plt.plot(dimrange[:3], np.median(dpp_times[:, :3, 2], axis=0), label="dpp, N:{0}".format(dimrange[2]))

# plt.yscale("log")
plt.ylim(ymax=10)
plt.xticks(dimrange)
plt.legend()

plt.ylabel("execution times (s)", fontsize=15)
plt.show()

In [ ]:
with open("data/execution_times.pickle", "rb") as fp:
    tims_ = pickle.load(fp)
    
dpp_times = tims_["dpp"]
strauss_times = tims_["strauss"]